# Load dataset

加载数据集，转换为trl reward trainer兼容的格式

数据集转换的方式可以参考hf上的教程：https://huggingface.co/docs/trl/main/en/reward_trainer

In [15]:
from datasets import DatasetDict, load_dataset

dataset_path = "./COIG-P/data/*.parquet"

def make_conversation(example):
    prompt = example.get("conversations")[0]["value"]
    chosen = example.get("chosen")["value"]
    rejected = example.get("rejected")["value"]

    return {
        "chosen": [{"role": "user", "content": prompt}, {"role": "assistant", "content": chosen}],
        "rejected": [{"role": "user", "content": prompt}, {"role": "assistant", "content": rejected}],
    }


In [16]:
# 原始数据集
dataset = load_dataset("parquet", data_files=dataset_path)
print(dataset["train"][0])

{'conversations': [{'from': 'human', 'value': 'A、B两地分别在河流的上游和下游，相距18千米，甲、乙在静水中划船的速度分别为每小时12千米和9千米。同时岸上有一条公路连接A、B两地，长度为27千米，甲从A地出发沿公路开车到B地后立刻划船返回A地，用时2小时45分钟，乙从A地出发划船到B地后立即沿公路开车返回A地，用时2小时30分钟。则乙开车的速度为每小时多少千米？（两人换乘的时间均忽略不计）'}], 'chosen': {'from': 'gpt', 'value': '。\n\n设乙开车的速度为v千米/小时。\n\n甲从A地出发，先开车到B地，再划船返回A地，总用时2小时45分钟，即2.75小时。由于A、B两地之间的公路长度为27千米，河流长度为18千米，甲划船的速度为12千米/小时。\n\n设甲开车的速度为\\(v_1\\)千米/小时（为求解乙的速度，\\(v_1\\)为中间变量，但我们可以利用题设信息直接求出其值，不过此处先假设），则有：\n\n\\[v_1 \\times \\frac{27}{v_1} + 12 \\times \\frac{18}{12} = 2.75\\]\n\n这里的\\(\\frac{27}{v_1}\\)表示甲开车从A到B的时间，\\(\\frac{18}{12}\\)表示甲划船从B到A的时间，等号左边表示总时间。\n\n由于\\(\\frac{18}{12} = 1.5\\)小时，将之代入方程中解得甲开车的时间为\\(2.75 - 1.5 = 1.25\\)小时，进而解得\\(v_1 = \\frac{27}{1.25} = 21.6\\)千米/小时。这是甲开车的速度。\n\n乙从A地出发划船到B地，再开车返回A地，总用时2小时30分钟，即2.5小时。乙划船的速度为9千米/小时，乙划船从A到B的时间为\\(\\frac{18}{9} = 2\\)小时。\n\n设乙开车的速度为v千米/小时，则有：\n\n\\[2 + \\frac{27}{v} = 2.5\\]\n\n即\n\n\\[\\frac{27}{v} = 0.5\\]\n\n解此方程得乙开车的速度v为：\n\n\\[v = \\frac{27}{0.5} = 54\\]\n\n因此，乙开车的速度为每小时54千米。'}, '

In [17]:
# 转换数据集
dataset = dataset.map(make_conversation)
print(dataset["train"][0])

{'conversations': [{'from': 'human', 'value': 'A、B两地分别在河流的上游和下游，相距18千米，甲、乙在静水中划船的速度分别为每小时12千米和9千米。同时岸上有一条公路连接A、B两地，长度为27千米，甲从A地出发沿公路开车到B地后立刻划船返回A地，用时2小时45分钟，乙从A地出发划船到B地后立即沿公路开车返回A地，用时2小时30分钟。则乙开车的速度为每小时多少千米？（两人换乘的时间均忽略不计）'}], 'chosen': [{'content': 'A、B两地分别在河流的上游和下游，相距18千米，甲、乙在静水中划船的速度分别为每小时12千米和9千米。同时岸上有一条公路连接A、B两地，长度为27千米，甲从A地出发沿公路开车到B地后立刻划船返回A地，用时2小时45分钟，乙从A地出发划船到B地后立即沿公路开车返回A地，用时2小时30分钟。则乙开车的速度为每小时多少千米？（两人换乘的时间均忽略不计）', 'role': 'user'}, {'content': '。\n\n设乙开车的速度为v千米/小时。\n\n甲从A地出发，先开车到B地，再划船返回A地，总用时2小时45分钟，即2.75小时。由于A、B两地之间的公路长度为27千米，河流长度为18千米，甲划船的速度为12千米/小时。\n\n设甲开车的速度为\\(v_1\\)千米/小时（为求解乙的速度，\\(v_1\\)为中间变量，但我们可以利用题设信息直接求出其值，不过此处先假设），则有：\n\n\\[v_1 \\times \\frac{27}{v_1} + 12 \\times \\frac{18}{12} = 2.75\\]\n\n这里的\\(\\frac{27}{v_1}\\)表示甲开车从A到B的时间，\\(\\frac{18}{12}\\)表示甲划船从B到A的时间，等号左边表示总时间。\n\n由于\\(\\frac{18}{12} = 1.5\\)小时，将之代入方程中解得甲开车的时间为\\(2.75 - 1.5 = 1.25\\)小时，进而解得\\(v_1 = \\frac{27}{1.25} = 21.6\\)千米/小时。这是甲开车的速度。\n\n乙从A地出发划船到B地，再开车返回A地，总用时2小时30分钟，即2.5小时。乙划船的速度为9千米/小时，乙划船从A到B的时间

In [ ]:
# 划分数据集
split_dataset = dataset["train"].train_test_split(test_size=0.2, seed=42)
dataset = DatasetDict({
    "train": split_dataset["train"],
    "test": split_dataset["test"]
})
print(dataset["train"][0])
print(dataset["test"][0])

# Training

In [22]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments

# 加载模型
model_path = "./Qwen2.5-0.5B"
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=1)
tokenizer = AutoTokenizer.from_pretrained(model_path)
if tokenizer.pad_token is None:
    tokenizer.pad_token  = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id

# 加载配置
training_args = TrainingArguments(
    output_dir = "./Qwen2.5-0.5B-Reward",
    num_train_epochs=1,
    learning_rate=1.0e-5,
    per_device_train_batch_size= 8,
    eval_strategy="steps",
    eval_steps=50,
    gradient_checkpointing=True,
    logging_steps=1,
    report_to=None,
)

Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at ./Qwen2.5-0.5B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch
from accelerate import logging
from trl import RewardTrainer

trainer = RewardTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
)

trainer.train()